In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.utils.class_weight import compute_class_weight
import itertools
from IPython.display import display

%matplotlib inline

In [2]:
x_train = pd.read_csv('data/s2e/audio_train.csv')
x_test = pd.read_csv('data/s2e/audio_test.csv')

print(x_train.shape)
y_train = x_train['label']
y_test = x_test['label']

(7837, 10)


In [3]:
print(x_train.shape, x_test.shape)
cl_weight = dict(pd.Series(x_train['label']).value_counts(normalize=True))
print(dict(pd.Series(x_train['label']).value_counts()))

del x_train['label']
del x_test['label']
del x_train['wav_file']
del x_test['wav_file']

(7837, 10) (1960, 10)
{0: 879, 1: 1325, 2: 2334, 3: 1010, 4: 937, 5: 1352}


In [4]:
emotion_dict = {'ang': 0,
                'hap': 1,
                'sad': 2,
                'fea': 3,
                'sur': 4,
                'neu': 5}

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    # plt.figure(figsize=(8,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
def one_hot_encoder(true_labels, num_records, num_classes):
    temp = np.array(true_labels[:num_records])
    true_labels = np.zeros((num_records, num_classes))
    true_labels[np.arange(num_records), temp] = 1
    return true_labels

def display_results(y_test, pred_probs, cm=True):
    pred = np.argmax(pred_probs, axis=-1)
    one_hot_true = one_hot_encoder(y_test, len(pred), len(emotion_dict))
    print('Test Set Accuracy =  {0:.3f}'.format(accuracy_score(y_test, pred)))
    print('Test Set F-score =  {0:.3f}'.format(f1_score(y_test, pred, average='macro')))
    print('Test Set Precision =  {0:.3f}'.format(precision_score(y_test, pred, average='macro')))
    print('Test Set Recall =  {0:.3f}'.format(recall_score(y_test, pred, average='macro')))
    if cm:
        plot_confusion_matrix(confusion_matrix(y_test, pred), classes=emotion_dict.keys())

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=1200, min_samples_split=25)
rf_classifier.fit(x_train, y_train)

# Predict
pred_probs = rf_classifier.predict_proba(x_test)

# Results
display_results(y_test, pred_probs)

with open('pred_probas/rf_classifier.pkl', 'wb') as f:
    pickle.dump(pred_probs, f)

In [ ]:
xgb_classifier = xgb.XGBClassifier(max_depth=7, learning_rate=0.008, objective='multi:softprob', 
                                   n_estimators=1200, sub_sample=0.8, num_class=len(emotion_dict),
                                   booster='gbtree', n_jobs=4)
xgb_classifier.fit(x_train, y_train)

# Predict
pred_probs = xgb_classifier.predict_proba(x_test)

# Results
display_results(y_test, pred_probs)

with open('pred_probas/xgb_classifier.pkl', 'wb') as f:
    pickle.dump(pred_probs, f)

In [ ]:
mlp_classifier = MLPClassifier(hidden_layer_sizes=(650, ), activation='relu', solver='adam', alpha=0.0001,
                               batch_size='auto', learning_rate='adaptive', learning_rate_init=0.01,
                               power_t=0.5, max_iter=1000, shuffle=True, random_state=None, tol=0.0001,
                               verbose=False, warm_start=True, momentum=0.8, nesterovs_momentum=True,
                               early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999,
                               epsilon=1e-08)

mlp_classifier.fit(x_train, y_train)

# Predict
pred_probs = mlp_classifier.predict_proba(x_test)

# Results
display_results(y_test, pred_probs)

with open('pred_probas/mlp_classifier.pkl', 'wb') as f:
    pickle.dump(pred_probs, f)

In [ ]:
# Discard SVM and KNN as they don't perform well
# svm_classifier = SVC(C=5000.0, probability=True, kernel='rbf', gamma='scale')
# svm_classifier.fit(x_train, y_train)

# # Predict
# pred_probs = svm_classifier.predict_proba(x_test)

# # Results
# display_results(y_test, pred_probs)

# with open('pred_probas/svm_classifier.pkl', 'wb') as f:
#     pickle.dump(pred_probs, f)

In [ ]:
ax = xgb.plot_importance(xgb_classifier, max_num_features=10, height=0.5, show_values=False)
fig = ax.figure
fig.set_size_inches(8, 8)

In [ ]:
contribution_scores = xgb_classifier.feature_importances_
contribution_scores

In [ ]:
top_n = [3, 5, 6, 7, 8]  # number of features
for n in top_n:
    threshold = np.argsort(contribution_scores)[::-1][:n][-1]
    print('Stats for top {} features:'.format(n))
    # Select features using threshold
    selection = SelectFromModel(xgb_classifier, threshold=contribution_scores[threshold], prefit=True)
    select_x_train = selection.transform(x_train)
    select_x_test = selection.transform(x_test)
    
    # Train
    select_xgb_classifier = xgb.XGBClassifier(max_depth=7, learning_rate=0.008, objective='multi:softprob', 
                                              n_estimators=1200, sub_sample = 0.8, num_class = len(emotion_dict),
                                              booster='gbtree', n_jobs=4)
    select_xgb_classifier.fit(select_x_train, y_train)

    # Predict
    pred_probs = select_xgb_classifier.predict_proba(select_x_test)

    # Results
    display_results(y_test, pred_probs, cm = False)

In [5]:
import torch
import pickle
import numpy as np
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from lstm_classifier.s2e.lstm_classifier import LSTMClassifier
from lstm_classifier.s2e import utils
from lstm_classifier.s2e.config import model_config as config

emotion_dict = {'ang': 0, 'hap': 1, 'sad': 2, 'fea': 3, 'sur': 4, 'neu': 5}

device = 'cuda:{}'.format(config['gpu']) if \
         torch.cuda.is_available() else 'cpu'

model = LSTMClassifier(config)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

train_batches = [[torch.FloatTensor(np.array(x_train)), torch.LongTensor(np.array(y_train))]]
test_pairs = [torch.FloatTensor(np.array(x_test)), torch.LongTensor(np.array(y_test))]

best_perf = {}
best_acc = 0

print(config['n_epochs'])
for epoch in range(config['n_epochs']):
    losses = []
    for batch in train_batches:
        inputs = batch[0].unsqueeze(0)  # frame in format as expected by model
        targets = batch[1]
        inputs = inputs.to(device)
        targets = targets.to(device)

        model.zero_grad()
        optimizer.zero_grad()

        predictions = model(inputs)
        predictions = predictions.to(device)

        loss = criterion(predictions, targets)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

    # evaluate
    with torch.no_grad():
        inputs = test_pairs[0].unsqueeze(0)
        targets = test_pairs[1]

        inputs = inputs.to(device)
        preds = model(inputs)
#         predictions = torch.argmax(model(inputs), dim=1)  # take argmax to get class id
#         predictions = predictions.to(device)
#         predictions = np.array(predictions.cpu())

        # evaluate on cpu
        targets = np.array(targets.cpu())
        preds = np.array(preds.cpu().numpy())
        predictions = np.argmax(preds, axis=-1)

        # Get results
        # plot_confusion_matrix(targets, predictions,
        #                       classes=emotion_dict.keys())
        performance = utils.evaluate(targets, predictions)
        
        # Save
        if performance['acc'] > best_acc:
            best_perf = performance
            best_acc = performance['acc']
            with open('pred_probas/lstm_classifier.pkl', 'wb') as f:
                pickle.dump(preds, f)
print(best_perf)

85000


/home/gsahu/dev/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/gsahu/dev/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


{'precision': 0.48196378854959504, 'f1': 0.3224630221583152, 'recall': 0.31405077581162466, 'acc': 0.38010204081632654}


In [ ]:
# Load predicted probabilities
with open('pred_probas/xgb_classifier.pkl', 'rb') as f:
    xgb_pred_probs = pickle.load(f)
    
with open('pred_probas/svm_classifier.pkl', 'rb') as f:
    svm_pred_probs = pickle.load(f)
    
with open('pred_probas/rf_classifier.pkl', 'rb') as f:
    rf_pred_probs = pickle.load(f)
    
with open('pred_probas/mlp_classifier.pkl', 'rb') as f:
    mlp_pred_probs = pickle.load(f)
    
with open('pred_probas/lstm_classifier.pkl', 'rb') as f:
    lstm_pred_probs = pickle.load(f)
    
print(lstm_pred_probs[0])

n_samples = np.min([len(xgb_pred_probs), len(svm_pred_probs), len(rf_pred_probs), len(mlp_pred_probs), len(lstm_pred_probs)])

# Average of the predicted probabilites
ensemble_pred_probs = (lstm_pred_probs[:n_samples])/1.0
# ensemble_pred_probs = (xgb_pred_probs[:n_samples] + mlp_pred_probs[:n_samples] + rf_pred_probs[:n_samples])/3.0


# Show metrics
display_results(y_test[:n_samples], ensemble_pred_probs)